<center>
    <img src="https://www.clearlyrated.com/brand-logo/talent-path" width="500" alt="cognitiveclass.ai logo"  />
</center>

<h1 style="color:DarkBlue; text-align:center">Altair >> Advanced Topics</h1>
<p>
    In this workbook we will cover 3 main topics with an Activity at the end:
    <ul>
        <li><a href="#Compound-Charts">Compound Charts</a></li> 
        <li><a href="#Interactivity-and-Selections">Interactivity and Selections</a></li> 
        <li><a href="#Encodings-Cheat-Sheet">Encodings Cheat Sheet</a></li> 
        <li><a href="#Activity">Activity</a></li> 
    </ul>
</p>

# Compound Charts

Altair provides a concise API for creating multi-panel and layered charts, and we'll mention three of them explicitly here:

<ul>
        <li>Layering</li> 
        <li>Horizontal Concatenation</li> 
        <li>Vertical Concatenation</li> 
        <li>Repeat Charts</li> 
</ul>

In [1]:
import altair as alt
from vega_datasets import data

## Layering

Layering lets you put layer multiple marks on a single Chart. One common example is creating a plot with both points and lines representing the same data.

Let's use the ``stocks`` data for this example:

In [2]:
stocks = data.stocks()
stocks.head()

,symbol,date,price
0,MSFT,2000-01-01,39.81
1,MSFT,2000-02-01,36.35
2,MSFT,2000-03-01,43.22
3,MSFT,2000-04-01,28.37
4,MSFT,2000-05-01,25.45


Here is a simple line plot for the stocks data:

In [18]:
lines = alt.Chart(stocks).mark_line().encode(
    x = 'date:T',
    y = 'price:Q' ,
    color='symbol:N'
)
lines

alt.Chart(...)

and here is the same plot with a ``circle`` mark:

In [19]:
points = alt.Chart(stocks).mark_circle().encode(
    x = 'date:T',
    y = 'price:Q' ,
    color='symbol:N'
)
points

alt.Chart(...)

How do we get the plots to be Concatenated together?

In [20]:
lines + points

alt.LayerChart(...)

This ``+`` is just a shortcut to the ``alt.layer()`` function, which does the same thing:

In [21]:
alt.layer(points,lines)

alt.LayerChart(...)

One pattern we'll use often is to create a base chart with the common elements, and add together two copies with just a single change:

In [27]:
base = alt.Chart(stocks).encode(
    x='date:T',
    y='price:Q',
    color='symbol:N'
)
base.mark_line() + base.mark_circle()

alt.LayerChart(...)

## Horizontal Concatenation

Just as we can layer charts on top of each other, we can concatenate horizontally using ``alt.hconcat``, or equivalently the ``|`` operator:

In [28]:
alt.hconcat(base.mark_line(), base.mark_circle())

alt.HConcatChart(...)

In [31]:
alt.hconcat(base.mark_line(), base.mark_circle())

alt.HConcatChart(...)

This can be most useful for creating multi-panel views; for example, here is the iris dataset:

In [32]:
iris = data.iris()
iris.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [33]:
base = alt.Chart(iris).mark_point().encode(
    x = 'sepalLength',
    y = 'sepalWidth',
    color= 'species'
)
base | base.encode(x='petalLength')

alt.HConcatChart(...)

## Vertical Concatenation

Vertical concatenation looks a lot like horizontal concatenation, but using either the ``alt.hconcat()`` function, or the ``&`` operator:

In [34]:
base & base.encode(y='petalLength')

alt.VConcatChart(...)

## Repeat Charts
### Small Multiples (SPLOM)

Because it is such a common pattern to horizontally and vertically concatenate charts while changing one encoding, Altair offers a shortcut for this, using the ``repeat()`` operator.

In [37]:
iris = data.iris()
fields = ['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth']

alt.Chart(iris).mark_point().encode(
    x = alt.X(alt.repeat('column'),type='quantitative'),
    y = alt.Y(alt.repeat('row'),type='quantitative'),
    color='species'
).properties(
    width=200,
    height=200
).repeat(
    row=fields,
    column=fields[::-1] # Steps backwards
)

alt.RepeatChart(...)

### US Population: Wrapped Facet

This chart visualizes the age distribution of the US population over time, using a wrapped faceting of the data by decade.

In [39]:
population = data.population()

alt.Chart(population).mark_area(color = '#24ad15').encode(
    x='age:O',
    y=alt.Y(
        'sum(people):Q',
        title='Population',
        axis=alt.Axis(format='~s')
    ),
    facet=alt.Facet('year:O', columns=5)
).properties(
    title='US Age Distribution By Year',
    width=90,
    height=80
)

alt.Chart(...)

# Interactivity and Selections

Altair's interactivity and grammar of selections are one of its unique features among available plotting libraries.
In this section, we will walk through the variety of selection types that are available, and begin to practice creating interactive charts and dashboards.

There are three basic types of selections available:

- Interval Selection: ``alt.selection_interval()``
- Single Selection: ``alt.selection_single()``
- Multi Selection: ``alt.selection_multi()``

And we will cover four basic things that you can do with these selections
<ul>
        <li>Conditional encodings</li> 
        <li>Scales</li> 
        <li>Filters</li> 
        <li>Domains</li> 
</ul>

## Basic Interactions: Panning, Zooming, Tooltips

The basic interactions that Altair makes available are panning, zooming, and tooltips.
This can be done in your chart without any use of the selection interface, using the
``interactive()`` shortcut method and the ``tooltip`` encoding.

For example, with our standard cars dataset, we can do the following:

In [40]:
cars = data.cars()
cars.head()

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


In [42]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin',
    tooltip=['Name']
).interactive()

alt.Chart(...)

At this point, hovering over a point will bring up a tooltip with the name of the car model, and clicking/dragging/scrolling will pan and zoom on the plot.

## More Sophisticated Interaction: Selections

### Basic Selection Example: Interval

As an example of a selection, let's add an interval selection to a chart.

We'll start with our cannonical scatter plot:

In [43]:
interval = alt.selection_interval()
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin',
    tooltip=['Name']
).add_selection(
    interval
)

alt.Chart(...)

To add selection behavior to a chart, we create the selection object and use the `add_selection` method:

This adds an interaction to the plot that lets us select points on the plot; perhaps the most common use of a selection is to highlight points by conditioning their color on the result of the selection.

This can be done with ``alt.condition``:

In [45]:
interval = alt.selection_interval()
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(interval, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    interval
)

alt.Chart(...)

The ``alt.condition`` function takes three arguments: a selection object, a value to be applied to points within the selection, and a value to be applied to points outside the selection.
Here we use ``alt.value('lightgray')`` to make certain that the color is treated as an actual color, rather than the name of a data column.

#### Customizing the Interval selection

The ``alt.selection_interval()`` function takes a number of additional arguments; for example, by specifying ``encodings``, we can control whether the selection covers x, y, or both:

In [46]:
interval = alt.selection_interval(encodings=['x'])
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(interval, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    interval
)

alt.Chart(...)

In [47]:
interval = alt.selection_interval(encodings=['y'])
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(interval, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    interval
)

alt.Chart(...)

The ``empty`` argument lets us control whether empty selections contain *all* values, or *none* of the values;
with ``empty='none'`` points are grayed-out by default:

In [50]:
interval = alt.selection_interval(empty='none')
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(interval, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    interval
)

alt.Chart(...)

### Single Selections

The ``alt.selection_single()`` function allows the user to click on single chart objects to select them, one at a time.
We'll make the points a bit bigger so they are easier to click:

In [52]:
single = alt.selection_single()

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(single, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    single
)

alt.Chart(...)

The single selection allows other behavior as well; for example, we can set ``nearest=True`` and ``on='mouseover'`` to update the highlight to the nearest point as we move the mouse:

In [53]:
single = alt.selection_single(on='mouseover',nearest=True)

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(single, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    single
)

alt.Chart(...)

### Multi Selection

The ``alt.selection_multi()`` function is quite similar to the ``single`` function, except it lets multiple points be selected at once, while holding the shift key:

In [62]:
multi = alt.selection_multi()
# Hold Shift
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(multi, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    multi
)

alt.Chart(...)

Options like ``on`` and ``nearest`` also work for multi selections:

In [63]:
multi = alt.selection_multi(on='mouseover')
# Hold shift to multiselect
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(multi, 'Origin', alt.value('lightgray')),
    tooltip=['Name']
).add_selection(
    multi
)

alt.Chart(...)

## Selection Binding

Above we have seen how ``alt.condition`` can be used to bind the selection to different aspects of the chart.
Let's look at a few other ways that a selection can be used:

### Binding Scales
For an interval selection, another thing you can do with the selection is bind the selection region to the chart scales:

In [65]:
bind = alt.selection_interval(bind='scales')

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    tooltip=['Name']
).add_selection(
    bind
)

alt.Chart(...)

This is essentially what the ``chart.interactive()`` method does under the hood.

### Binding Scales to Other Domains

It is also possible to bind scales to other domains, which can be useful in creating

In [66]:
weather = data.seattle_weather()
weather.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [75]:
base = alt.Chart(weather).mark_rule().encode(
    x = 'date:T',
    y= 'temp_min:Q',
    y2= 'temp_max:Q',
    color= 'weather:N'
)
base

alt.Chart(...)

In [72]:
base = alt.Chart(weather[:1]).mark_rule().encode(
    x = 'date:T',
    y= 'temp_min:Q',
    y2= 'temp_max:Q',
    color= 'weather:N'
)
base

alt.Chart(...)

In [76]:
chart = base.properties(
    width= 800,
    height= 300
)
view = chart.properties(
    width = 800,
    height = 50
)
chart & view

alt.VConcatChart(...)

Let's add an interval selection to the bottom chart that will control the domain of the top chart:

In [81]:
interval = alt.selection_interval(encodings=['x'])
base = alt.Chart(weather).mark_rule().encode(
    x = 'date:T',
    y= 'temp_min:Q',
    y2= 'temp_max:Q',
    color= 'weather:N'
)
chart = base.encode(
    x=alt.X('date:T', scale=alt.Scale(domain=interval.ref()))
).properties(
    width= 800,
    height= 300
)

view = base.add_selection(
    interval
).properties(
    width = 800,
    height = 50
)
chart & view

alt.VConcatChart(...)

### Filtering by Selection / Brushing and Linking

In multi-panel charts, we can use the result of the selection to filter other views of the data.
For example, here is a scatter-plot along with a histogram

In [84]:
scatter = alt.Chart(cars).mark_point().encode(
    x = 'Horsepower',
    y = 'Miles_per_Gallon:Q',
    color = 'Origin:N'
)
hist = alt.Chart(cars).mark_bar().encode(
    x= 'count()',
    y='Origin:N',
    color='Origin:N'
)
scatter & hist

alt.VConcatChart(...)

Similarly, you can use a Multi selection to go the other way (allow clicking on the bar chart to filter the contents of the scatter plot.
We'll add this to the previous chart:

In [87]:
interval = alt.selection_interval()

scatter = alt.Chart(cars).mark_point().encode(
    x = 'Horsepower',
    y = 'Miles_per_Gallon:Q',
    color = alt.condition(interval,'Origin:N',alt.value('lightgray'))
).add_selection(
    interval
)


hist = alt.Chart(cars).mark_bar().encode(
    x= 'count()',
    y='Origin:N',
    color='Origin:N'
).transform_filter(
    interval
)
scatter & hist

alt.VConcatChart(...)

In [89]:
click = alt.selection_multi(encodings=['color'])

scatter = alt.Chart(cars).mark_point().encode(
    x = 'Horsepower',
    y = 'Miles_per_Gallon:Q',
    color = 'Origin:N'
).transform_filter(
    click
)

hist = alt.Chart(cars).mark_bar().encode(
    x= 'count()',
    y='Origin:N',
    color= alt.condition(click, 'Origin:N', alt.value('lightgrey'))
).add_selection(
    click
)
scatter & hist

alt.VConcatChart(...)

In [91]:
click = alt.selection_multi(encodings=['color'])

scatter = alt.Chart(cars).mark_point().encode(
    x = 'Horsepower:Q',
    y = alt.Y('Miles_per_Gallon:Q',scale=alt.Scale(domain=[0,50])),
    color = 'Origin:N'
).properties(
    width=500,
    height=300
).transform_filter(
    click
)

hist = alt.Chart(cars).mark_bar().encode(
    x= 'count()',
    y='Origin:N',
    color= alt.condition(click, 'Origin:N', alt.value('lightgrey'))
).add_selection(
    click
)
scatter & hist

alt.VConcatChart(...)

## Selections Summary

- Selection Types:

  - ``selection_interval()``
  - ``selection_single()``
  - ``selection_multi()``
  
- Bindings

  - bind scales: drag & scroll to interact with plot
  - bind scales on another chart
  - conditional encodings (e.g. color, size)
  - filter data

<hr>
<h1 style='text-align:Center' >Encodings Cheat Sheet</h1>


<table border="1" class="docutils">
    <caption style='font-size: 20px'>Data Types</caption>
    <colgroup>
        <col width="16%">
        <col width="19%">
        <col width="65%">
    </colgroup>
    <thead valign="bottom">
        <tr class="row-odd">
            <th class="head">Data Type</th>
            <th class="head">Shorthand Code</th>
            <th class="head">Description</th>
        </tr>
    </thead>
    <tbody valign="top">
        <tr class="row-even"><td>quantitative</td>
            <td><code class="docutils literal"><span class="pre">Q</span></code></td>
            <td>a continuous real-valued quantity</td>
        </tr>
        <tr class="row-odd"><td>ordinal</td>
            <td><code class="docutils literal"><span class="pre">O</span></code></td>
            <td>a discrete ordered quantity</td>
        </tr>
        <tr class="row-even"><td>nominal</td>
            <td><code class="docutils literal"><span class="pre">N</span></code></td>
            <td>a discrete unordered category</td>
        </tr>
        <tr class="row-odd"><td>temporal</td>
            <td><code class="docutils literal"><span class="pre">T</span></code></td>
            <td>a time or date value</td>
        </tr>
        <tr class="row-even"><td>geojson</td>
            <td><code class="docutils literal"><span class="pre">G</span></code></td>
            <td>a geographic shape</td>
        </tr>
    </tbody>
</table>

<table border="1" class="docutils">
    <caption style='font-size: 20px'>Position Channels</caption>
    <colgroup>
        <col width="16%">
        <col width="19%">
        <col width="65%">
    </colgroup>
    <thead valign="bottom">
        <tr class="row-odd"><th class="head">Channel</th>
            <th class="head">Altair Class</th>
            <th class="head">Description</th>
        </tr>
    </thead>
    <tbody valign="top">
        <tr class="row-even"><td>x</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.X.html#altair.X" title="altair.X"><code class="xref py py-class docutils literal"><span class="pre">X</span></code></a></td>
            <td>The x-axis value</td>
        </tr>
        <tr class="row-odd"><td>y</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Y.html#altair.Y" title="altair.Y"><code class="xref py py-class docutils literal"><span class="pre">Y</span></code></a></td>
            <td>The y-axis value</td>
        </tr>
        <tr class="row-even"><td>x2</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.X2.html#altair.X2" title="altair.X2"><code class="xref py py-class docutils literal"><span class="pre">X2</span></code></a></td>
            <td>Second x value for ranges</td>
        </tr>
        <tr class="row-odd"><td>y2</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Y2.html#altair.Y2" title="altair.Y2"><code class="xref py py-class docutils literal"><span class="pre">Y2</span></code></a></td>
            <td>Second y value for ranges</td>
        </tr>
        <tr class="row-even"><td>longitude</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Longitude.html#altair.Longitude" title="altair.Longitude"><code class="xref py py-class docutils literal"><span class="pre">Longitude</span></code></a></td>
            <td>Longitude for geo charts</td>
        </tr>
        <tr class="row-odd"><td>latitude</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Latitude.html#altair.Latitude" title="altair.Latitude"><code class="xref py py-class docutils literal"><span class="pre">Latitude</span></code></a></td>
            <td>Latitude for geo charts</td>
        </tr>
        <tr class="row-even"><td>longitude2</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Longitude2.html#altair.Longitude2" title="altair.Longitude2"><code class="xref py py-class docutils literal"><span class="pre">Longitude2</span></code></a></td>
            <td>Second longitude value for ranges</td>
        </tr>
        <tr class="row-odd"><td>latitude2</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Latitude2.html#altair.Latitude2" title="altair.Latitude2"><code class="xref py py-class docutils literal"><span class="pre">Latitude2</span></code></a></td>
            <td>Second latitude value for ranges</td>
        </tr>
        <tr class="row-even"><td>xError</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.XError.html#altair.XError" title="altair.XError"><code class="xref py py-class docutils literal"><span class="pre">XError</span></code></a></td>
            <td>The x-axis error value</td>
        </tr>
        <tr class="row-odd"><td>yError</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.YError.html#altair.YError" title="altair.YError"><code class="xref py py-class docutils literal"><span class="pre">YError</span></code></a></td>
            <td>The y-axis error value</td>
        </tr>
        <tr class="row-even"><td>xError2</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.XError2.html#altair.XError2" title="altair.XError2"><code class="xref py py-class docutils literal"><span class="pre">XError2</span></code></a></td>
            <td>The second x-axis error value</td>
        </tr>
        <tr class="row-odd"><td>yError2</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.YError2.html#altair.YError2" title="altair.YError2"><code class="xref py py-class docutils literal"><span class="pre">YError2</span></code></a></td>
            <td>The second y-axis error value</td>
        </tr>
    </tbody>
</table>

<table border="1" class="docutils">
    <caption style='font-size: 20px'>Mark Property Channels</caption>
    <colgroup>
        <col width="16%">
        <col width="19%">
        <col width="65%">
    </colgroup>
    <thead valign="bottom">
        <tr class="row-odd">
            <th class="head">Channel</th>
            <th class="head">Altair Class</th>
            <th class="head">Description</th>
        </tr>
    </thead>
    <tbody valign="top">
        <tr class="row-even"><td>color</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/core/altair.Color.html#altair.Color" title="altair.Color"><code class="xref py py-class docutils literal"><span class="pre">Color</span></code></a></td>
            <td>The color of the mark</td>
        </tr>
        <tr class="row-odd"><td>fill</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Fill.html#altair.Fill" title="altair.Fill"><code class="xref py py-class docutils literal"><span class="pre">Fill</span></code></a></td>
            <td>The fill for the mark</td>
        </tr>
        <tr class="row-even"><td>fillopacity</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.FillOpacity.html#altair.FillOpacity" title="altair.FillOpacity"><code class="xref py py-class docutils literal"><span class="pre">FillOpacity</span></code></a></td>
            <td>The opacity of the mark’s fill</td>
        </tr>
        <tr class="row-odd"><td>opacity</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Opacity.html#altair.Opacity" title="altair.Opacity"><code class="xref py py-class docutils literal"><span class="pre">Opacity</span></code></a></td>
            <td>The opacity of the mark</td>
        </tr>
        <tr class="row-even"><td>shape</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Shape.html#altair.Shape" title="altair.Shape"><code class="xref py py-class docutils literal"><span class="pre">Shape</span></code></a></td>
            <td>The shape of the mark</td>
        </tr>
        <tr class="row-odd"><td>size</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Size.html#altair.Size" title="altair.Size"><code class="xref py py-class docutils literal"><span class="pre">Size</span></code></a></td>
            <td>The size of the mark</td>
        </tr>
        <tr class="row-even"><td>stroke</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Stroke.html#altair.Stroke" title="altair.Stroke"><code class="xref py py-class docutils literal"><span class="pre">Stroke</span></code></a></td>
            <td>The stroke of the mark</td>
        </tr>
        <tr class="row-odd"><td>strokeDash</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.StrokeDash.html#altair.StrokeDash" title="altair.StrokeDash"><code class="xref py py-class docutils literal"><span class="pre">StrokeDash</span></code></a></td>
            <td>The stroke dash style</td>
        </tr>
        <tr class="row-even"><td>strokeOpacity</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.StrokeOpacity.html#altair.StrokeOpacity" title="altair.StrokeOpacity"><code class="xref py py-class docutils literal"><span class="pre">StrokeOpacity</span></code></a></td>
            <td>The opacity of the line</td>
        </tr>
        <tr class="row-odd"><td>strokeWidth</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.StrokeWidth.html#altair.StrokeWidth" title="altair.StrokeWidth"><code class="xref py py-class docutils literal"><span class="pre">StrokeWidth</span></code></a></td>
            <td>The width of the line</td>
        </tr>
    </tbody>
</table>

<table border="1" class="docutils">
    <caption style='font-size: 20px'>Text and Tooltip Channels</caption>
    <colgroup>
        <col width="16%">
        <col width="19%">
        <col width="65%">
    </colgroup>
    <thead valign="bottom">
        <tr class="row-odd">
            <th class="head">Channel</th>
            <th class="head">Altair Class</th>
            <th class="head">Description</th>
        </tr>
    </thead>
    <tbody valign="top">
        <tr class="row-even"><td>text</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/core/altair.Text.html#altair.Text" title="altair.Text"><code class="xref py py-class docutils literal"><span class="pre">Text</span></code></a></td>
            <td>Text to use for the mark</td>
        </tr>
        <tr class="row-odd"><td>key</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Key.html#altair.Key" title="altair.Key"><code class="xref py py-class docutils literal"><span class="pre">Key</span></code></a></td>
            <td>–</td>
        </tr>
        <tr class="row-even"><td>tooltip</td>
            <td><a class="reference internal" href="https://altair-viz.github.io/user_guide/generated/channels/altair.Tooltip.html#altair.Tooltip" title="altair.Tooltip"><code class="xref py py-class docutils literal"><span class="pre">Tooltip</span></code></a></td>
            <td>The tooltip value</td>
        </tr>
    </tbody>
</table>

<hr>

# Activity


## Compound Charts Activity
Link to this section: <a href="#Compound-Charts">Compound Charts</a>
1. Create horizontally Juxtaposed(concattenated) bar charts using the population data.  The left bar chart should show the age and number of people in 1950 that are women (`sex` == 1) as a function of age, and the right bar chart should show the number of people in 1950 that are men (`sex` == 2) as a function of age. Make the left bar chart be the color '#F4D03F' and the right bar chart to be the color '#7D3C98'.  Filter the data using the slicing techniques in pandas.
2. Create a layered line graph that shows the number of women in 1950 as a function of age and the number of men in 1950 as a function of age. Use the same colors as the first problem.  It's ok for now if you don't have a legend (legends in layered altair charts are complicated).  Again, use the pandas slicing to filter the data prior to creating the chart. 

In [94]:
population = data.population()
population

,year,age,sex,people
0,1850,0,1,1483789
1,1850,0,2,1450376
2,1850,5,1,1411067
3,1850,5,2,1359668
4,1850,10,1,1260099
...,...,...,...,...
565,2000,80,2,3221898
566,2000,85,1,970357
567,2000,85,2,1981156
568,2000,90,1,336303


In [114]:
newdf = df[df[population]]

alt.Chart(...)

## Interactivity and Selections
Link to this section: <a href="#Interactivity-and-Selections">Interactivity and Selections</a>

1. Using the cars data, create a scatter-plot of horsepower vs miles per gallon where the *size* of the points becomes larger as you hover over them.   
(Hint: use a condition inside of size)

2. Using the cars data, create a two-panel histogram (miles per gallon counts in one panel, horsepower counts in the other) where you can drag your mouse to select data in the left panel to filter the data in the second panel.
